# Cargamos los Datasets

In [1]:
import pandas as pd
import os
import numpy as np

# Directorio donde se encuentran los datasets
directory = 'datasets/'

# Diccionario para almacenar los DataFrames
dataframes = {}

# Cargar todos los datasets
for filename in os.listdir(directory):
    if filename.endswith('_historical_data.csv'):
        # Extraer el nombre de la empresa del nombre del archivo
        company_name = filename.replace('_historical_data.csv', '')
        
        # Ruta completa del archivo
        file_path = os.path.join(directory, filename)
        
        # Cargar el CSV en un DataFrame
        df = pd.read_csv(file_path)
        
        # Almacenar el DataFrame en el diccionario
        dataframes[company_name] = df
        
        print(f"Cargado: {filename}")
        print(f"Forma del DataFrame: {df.shape}")
        print(f"Columnas: {df.columns.tolist()}")
        print("------------------------")

# Imprimir los nombres de las empresas cargadas
print("DataFrames cargados:")
for company in dataframes.keys():
    print(company)



Cargado: ypf_historical_data.csv
Forma del DataFrame: (674, 8)
Columnas: ['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits']
------------------------
Cargado: pampa_energia_historical_data.csv
Forma del DataFrame: (674, 8)
Columnas: ['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits']
------------------------
Cargado: ecopetrol_historical_data.csv
Forma del DataFrame: (674, 8)
Columnas: ['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits']
------------------------
Cargado: petrobras_historical_data.csv
Forma del DataFrame: (674, 8)
Columnas: ['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits']
------------------------
Cargado: interconexion_electrica_historical_data.csv
Forma del DataFrame: (674, 8)
Columnas: ['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits']
------------------------
Cargado: eletrobras_historical_data.csv
Forma del DataFrame: (674, 8

# Exploración de los datos

Advertimos que la columna Date se encuentra en formato object.

In [2]:
# Aplicamos la función .info() a todos los DataFrames
for company, df in dataframes.items():
    print(f"\nInformación para {company}:")
    print(df.info())
    print("-" * 50)


Información para ypf:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 674 entries, 0 to 673
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Date          674 non-null    object 
 1   Open          674 non-null    float64
 2   High          674 non-null    float64
 3   Low           674 non-null    float64
 4   Close         674 non-null    float64
 5   Volume        674 non-null    int64  
 6   Dividends     674 non-null    float64
 7   Stock Splits  674 non-null    float64
dtypes: float64(6), int64(1), object(1)
memory usage: 42.3+ KB
None
--------------------------------------------------

Información para pampa_energia:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 674 entries, 0 to 673
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Date          674 non-null    object 
 1   Open          674 non-null    float64
 2   High      

### Explicación del contenido de las columnas
* 'Date': La fecha de la información registrada.
* 'Open': El precio de apertura de la acción al inicio del día de negociación.
* 'High': El precio más alto alcanzado por la acción durante el día.
* 'Low': El precio más bajo alcanzado por la acción durante el día.
* 'Close': El precio de cierre de la acción al final del día de negociación.
* 'Volume': El número total de acciones negociadas durante el día.
* 'Dividends': Los dividendos pagados por acción, si los hubiera.
* 'Stock Splits': Divisiones de acciones, si ocurrieron.

## Transformación columna Date

### Función para extraer solo la fecha (YYYY-MM-DD)

In [3]:
def transform_date(df):
    def extract_date(date_str):
        return date_str.split()[0]
    
    # Aplicar la función a la columna 'Date'
    df['Date'] = df['Date'].apply(extract_date)
    return df

# Aplicar la transformación a todos los dataframes
for company, df in dataframes.items():
    dataframes[company] = transform_date(df)



### Convertir la columna 'Date' a datetime

In [4]:
def convert_date_column(df):
    df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d')
    return df

# Aplicar la conversión a todos los dataframes
for company, df in dataframes.items():
    dataframes[company] = convert_date_column(df)


### Creamos tres nuevas columnas a partir de la columna Date: Year, Month y Day

In [5]:
def add_date_columns(df):
    # Asegurarse de que 'Date' es datetime
    df['Date'] = pd.to_datetime(df['Date'])
    
    # Crear nuevas columnas
    df['Year'] = df['Date'].dt.year
    df['Month'] = df['Date'].dt.month
    df['Day'] = df['Date'].dt.day
    
    return df

# Aplicar la función a todos los dataframes
for company, df in dataframes.items():
    dataframes[company] = add_date_columns(df)

# Utilizamos la biblioteca TA-Lib (Technical Analysis Library) cuyo propósito principal es proporcionar funciones para el análisis técnico de datos de series temporales

### Resumen de los indicadores a utilizar:

* SMA (Simple Moving Average):
El SMA es uno de los indicadores más básicos y ampliamente utilizados en el análisis técnico. Calcula el promedio aritmético de los precios de cierre durante un período específico. En nuestro código, utilizamos SMA de 50 y 200 días, que son períodos comúnmente usados para identificar tendencias a largo plazo.

* EMA (Exponential Moving Average):
El EMA es similar al SMA, pero da más peso a los datos más recientes, lo que lo hace más sensible a los cambios de precio recientes. En nuestro código, usamos un EMA de 20 días. El EMA reacciona más rápidamente a los cambios de precio que el SMA, lo que puede ser útil para identificar cambios de tendencia más temprano. 

* RSI (Relative Strength Index):
El RSI es un oscilador de momentum - es decir, la fuerza o velocidad con la que cambian dos valores extremos- que mide la velocidad y el cambio de los movimientos de precios. Oscila entre 0 y 100, y tradicionalmente, se considera que un RSI por encima de 70 indica una condición de sobrecompra, mientras que un RSI por debajo de 30 indica una condición de sobreventa. 

* MACD (Moving Average Convergence Divergence):
El MACD es un indicador de tendencia y momentum que muestra la relación entre dos EMAs, típicamente de 12 y 26 períodos. El indicador consta de tres componentes: la línea MACD (diferencia entre los dos EMAs), la línea de señal (generalmente un EMA de 9 períodos de la línea MACD) y el histograma (diferencia entre la línea MACD y la línea de señal). Los traders buscan cruces entre la línea MACD y la línea de señal como señales de compra o venta. 

* ATR (Average True Range):
El ATR es un indicador de volatilidad que mide la extensión media del movimiento de precios, independientemente de la dirección. No indica la dirección del precio, sino la volatilidad del mercado. Un ATR alto indica alta volatilidad, mientras que un ATR bajo indica baja volatilidad. Los traders utilizan el ATR para establecer stop-loss y take-profit, ajustando estos niveles en función de la volatilidad actual del mercado. 

* Stochastic Oscillator:
El Oscilador Estocástico es un indicador de momentum que compara el precio de cierre actual con el rango de precios durante un período específico. Consta de dos líneas: %K (la línea principal) y %D (la media móvil de %K). Oscila entre 0 y 100, con lecturas por encima de 80 consideradas como condiciones de sobrecompra y por debajo de 20 como condiciones de sobreventa. 

* OBV (On-Balance Volume):
El OBV es un indicador de volumen acumulativo que relaciona el volumen con los cambios de precio. La premisa es que los cambios en el OBV preceden a los cambios en el precio. Cuando el precio de cierre es mayor que el anterior, el volumen del día se suma al OBV; cuando es menor, se resta. Un OBV ascendente indica que el volumen está fluyendo hacia el activo (acumulación), mientras que un OBV descendente indica que el volumen está fluyendo fuera del activo (distribución). Los traders buscan divergencias entre el OBV y el precio como señales de posibles reversiones. 

In [6]:
from ta.trend import SMAIndicator, EMAIndicator, MACD, ADXIndicator
from ta.momentum import RSIIndicator, StochasticOscillator
from ta.volatility import AverageTrueRange
from ta.volume import OnBalanceVolumeIndicator
import numpy as np

def add_selected_indicators(df):
    # Asegurarse de que 'Date' es el índice y está ordenado
    df = df.set_index('Date').sort_index()
    
    # SMA para diferentes períodos
    df['SMA50'] = SMAIndicator(close=df['Close'], window=50).sma_indicator()
    df['SMA200'] = SMAIndicator(close=df['Close'], window=200).sma_indicator()
    
    # EMA
    df['EMA20'] = EMAIndicator(close=df['Close'], window=20).ema_indicator()
    
    # RSI
    df['RSI'] = RSIIndicator(close=df['Close']).rsi()
    
    # MACD
    macd = MACD(close=df['Close'])
    df['MACD'] = macd.macd()
    df['MACD_Signal'] = macd.macd_signal()
    df['MACD_Hist'] = macd.macd_diff()
    
    # ATR
    df['ATR'] = AverageTrueRange(high=df['High'], low=df['Low'], close=df['Close']).average_true_range()
    
    # Stochastic Oscillator
    stoch = StochasticOscillator(high=df['High'], low=df['Low'], close=df['Close'])
    df['Stoch_K'] = stoch.stoch()
    df['Stoch_D'] = stoch.stoch_signal()
    
    # OBV
    df['OBV'] = OnBalanceVolumeIndicator(close=df['Close'], volume=df['Volume']).on_balance_volume()
    
    # Volumen relativo
    df['Volume_SMA20'] = df['Volume'].rolling(window=20).mean()
    df['Relative_Volume'] = df['Volume'] / df['Volume_SMA20']
    
    # Rate of Change (ROC)
    df['ROC'] = (df['Close'] - df['Close'].shift(10)) / df['Close'].shift(10) * 100
    
    # Características temporales
    df['DayOfWeek'] = df.index.dayofweek
    df['Month'] = df.index.month
    
    # Retornos
    df['Daily_Return'] = df['Close'].pct_change()
    
    # Volatilidad histórica
    df['Volatility'] = df['Daily_Return'].rolling(window=20).std() * np.sqrt(252)
    
    # Eliminar filas con NaN que puedan haberse generado
    df = df.dropna()
    
    return df

# Aplicar la función a todos los dataframes
for company, df in dataframes.items():
    dataframes[company] = add_selected_indicators(df)

# Verificar el resultado
print(dataframes['ypf'].head())
print(dataframes['ypf'].columns)

            Open  High   Low  Close   Volume  Dividends  Stock Splits  Year  \
Date                                                                          
2022-10-18  7.10  7.19  6.76   6.94  1466300        0.0           0.0  2022   
2022-10-19  6.95  7.19  6.95   7.04  1570300        0.0           0.0  2022   
2022-10-20  7.15  7.49  7.02   7.05  2881100        0.0           0.0  2022   
2022-10-21  7.02  7.21  6.87   7.17  1743800        0.0           0.0  2022   
2022-10-24  7.10  7.61  6.82   7.39  5172100        0.0           0.0  2022   

            Month  Day  ...       ATR    Stoch_K    Stoch_D       OBV  \
Date                    ...                                             
2022-10-18     10   18  ...  0.403801  58.904117  55.479458  67047600   
2022-10-19     10   19  ...  0.392815  65.753426  64.155259  68617900   
2022-10-20     10   20  ...  0.398328  55.045899  59.901147  71499000   
2022-10-21     10   21  ...  0.394162  66.055061  62.284795  73242800   
2022-10-